<a href="https://colab.research.google.com/github/jasmine400/recommender/blob/main/question_similar_tst.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import pandas as pd
import numpy as np
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.metrics.pairwise import cosine_similarity

In [ ]:
!pip install -qU sentence-transformers
!pip install -qU wikipedia-api
!pip install -qU hazm
!pip install -qU clean-text[gpl]
!pip install -qU emoji

     |████████████████████████████████| 79 kB 2.6 MB/s 
     |████████████████████████████████| 3.8 MB 8.8 MB/s 
     |████████████████████████████████| 1.2 MB 32.7 MB/s 
     |████████████████████████████████| 67 kB 4.0 MB/s 
     |████████████████████████████████| 6.5 MB 14.7 MB/s 
     |████████████████████████████████| 596 kB 45.1 MB/s 
     |████████████████████████████████| 895 kB 43.5 MB/s 
     |████████████████████████████████| 316 kB 3.9 MB/s 
     |████████████████████████████████| 1.4 MB 50.4 MB/s 
     |████████████████████████████████| 233 kB 44.9 MB/s 
     |████████████████████████████████| 53 kB 1.2 MB/s 
     |████████████████████████████████| 175 kB 7.2 MB/s 
     |████████████████████████████████| 235 kB 54.1 MB/s 


In [ ]:
!mkdir resources
!wget -q "https://github.com/sobhe/hazm/releases/download/v0.5/resources-0.5.zip" -P resources
!unzip -qq resources/resources-0.5.zip -d resources

In [ ]:
!rm -rf /content/4ccae468eb73bf6c4f4de3075ddb5336
!rm -rf /content/preproc
!rm preprocessing.py utils.py
!mkdir -p /content/preproc
!git clone https://gist.github.com/4ccae468eb73bf6c4f4de3075ddb5336.git /content/preproc/
!mv /content/preproc/* /content/
!rm -rf /content/preproc

rm: cannot remove 'preprocessing.py': No such file or directory
rm: cannot remove 'utils.py': No such file or directory
Cloning into '/content/preproc'...
remote: Enumerating objects: 7, done.
remote: Total 7 (delta 0), reused 0 (delta 0), pack-reused 7
Unpacking objects: 100% (7/7), done.


In [ ]:
from transformers import BertConfig, BertTokenizer

from preprocessing import cleaning
from IPython import display

import numpy as np
import pandas as pd

import hazm
import requests
import time

import torch
from sentence_transformers import models, SentenceTransformer, util
from sklearn.cluster import KMeans

In [ ]:
def rtl_print(outputs, font_size="15px", n_to_br=False):
    outputs = outputs if isinstance(outputs, list) else [outputs]
    if n_to_br:
        outputs = [output.replace('\n', '<br/>') for output in outputs]

    outputs = [f'<p style="text-align: right; direction: rtl; margin-right: 10px; font-size: {font_size};">{output}</p>' for output in outputs]
    display.display(display.HTML(' '.join(outputs)))


def load_st_model(model_name_or_path):
    word_embedding_model = models.Transformer(model_name_or_path)
    pooling_model = models.Pooling(
        word_embedding_model.get_word_embedding_dimension(),
        pooling_mode_mean_tokens=True,
        pooling_mode_cls_token=False,
        pooling_mode_max_tokens=False)

    model = SentenceTransformer(modules=[word_embedding_model, pooling_model])
    return model

In [ ]:
# Load the Sentence-Transformer
embedder = load_st_model('m3hrdadfi/bert-fa-base-uncased-wikinli-mean-tokens')

Downloading:   0%|          | 0.00/519 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/621M [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/404 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/1.14M [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/112 [00:00<?, ?B/s]

In [ ]:
#confif bert model
config = BertConfig.from_pretrained(
    'm3hrdadfi/bert-fa-base-uncased-wikinli-mean-tokens', **{

    })
print(config.to_json_string())

{
  "_name_or_path": "HooshvareLab/bert-fa-base-uncased",
  "architectures": [
    "BertModel"
  ],
  "attention_probs_dropout_prob": 0.1,
  "classifier_dropout": null,
  "gradient_checkpointing": false,
  "hidden_act": "gelu",
  "hidden_dropout_prob": 0.1,
  "hidden_size": 768,
  "initializer_range": 0.02,
  "intermediate_size": 3072,
  "layer_norm_eps": 1e-12,
  "max_position_embeddings": 512,
  "model_type": "bert",
  "num_attention_heads": 12,
  "num_hidden_layers": 12,
  "pad_token_id": 0,
  "position_embedding_type": "absolute",
  "transformers_version": "4.17.0",
  "type_vocab_size": 2,
  "use_cache": true,
  "vocab_size": 100000
}



In [ ]:
corpus=[' دادن خمس سهام اهدایی به فرزند در زمان کودکی ']

In [ ]:
corpus_embeddings = embedder.encode(corpus, show_progress_bar=True)

Batches:   0%|          | 0/1 [00:00<?, ?it/s]

In [ ]:
#access google drive
from google.colab import drive
drive.mount('/content/gdrive')

Mounted at /content/gdrive


In [ ]:
#access google drive
%cd /content/gdrive/My\ Drive/parsa_clusters
!ls

/content/gdrive/My Drive/parsa_clusters
0.csv	    145.pickle	191.csv     236.pickle	282.csv     57.pickle
0.pickle    146.csv	191.pickle  237.csv	282.pickle  58.csv
100.csv     146.pickle	192.csv     237.pickle	283.csv     58.pickle
100.pickle  147.csv	192.pickle  238.csv	283.pickle  59.csv
101.csv     147.pickle	193.csv     238.pickle	284.csv     59.pickle
101.pickle  148.csv	193.pickle  239.csv	284.pickle  5.csv
102.csv     148.pickle	194.csv     239.pickle	285.csv     5.pickle
102.pickle  149.csv	194.pickle  23.csv	285.pickle  60.csv
103.csv     149.pickle	195.csv     23.pickle	286.csv     60.pickle
103.pickle  14.csv	195.pickle  240.csv	286.pickle  61.csv
104.csv     14.pickle	196.csv     240.pickle	287.csv     61.pickle
104.pickle  150.csv	196.pickle  241.csv	287.pickle  62.csv
105.csv     150.pickle	197.csv     241.pickle	288.csv     62.pickle
105.pickle  151.csv	197.pickle  242.csv	288.pickle  63.csv
106.csv     151.pickle	198.csv     242.pickle	289.csv     63.pickle
106.pickle 

In [ ]:
df = pd.read_csv('209.csv')

In [ ]:
df.head()

,Unnamed: 0,original_index,q&a,q_fa_id,question,clusters,row_num
0,2336,2336,یکی از دوستانم پتویی به خوابگاه دانشگاه سپرد ک...,1dlCAn8B7wsuAN6f4eP4,یکی از دوستانم پتویی به خوابگاه دانشگاه سپرد ک...,209,0
1,2391,2391,پدرم در یک سالن ورزشی کار می کند. استفاده شخصی...,1tlEAn8B7wsuAN6fBeSr,پدرم در یک سالن ورزشی کار می کند. استفاده شخصی...,209,1
2,2501,2501,مغازه‌ای اجاره کردم که دارای انشعاب آب است، ول...,jtlGAn8B7wsuAN6fw-f6,مغازه‌ای اجاره کردم که دارای انشعاب آب است، ول...,209,2
3,4399,4399,تکلیف پول جمع آوری شده توسط هیئتی که منحل شده...,F9kLAn8B7wsuAN6fu35u,تکلیف پول جمع آوری شده توسط هیئتی که منحل شده,209,3
4,5308,5308,کتاب امانتی که گم شده و نمی داند برای کدام کت...,ftkNAn8B7wsuAN6fFYIN,کتاب امانتی که گم شده و نمی داند برای کدام کت...,209,4


In [ ]:
print(len(df))

88


In [ ]:
import pickle

pickle_in=open('209.pickle','rb')
u=pickle.load(pickle_in)

In [ ]:
print(u)
print(u.shape)

[[-5.4695594e-01  3.0213717e-01  1.4343424e+00 ... -1.8749633e-01
  -6.4143288e-01 -4.6065682e-01]
 [ 2.2457790e-01  4.6777037e-01  4.0461478e-01 ... -3.7099230e-01
   2.6795432e-01 -9.3467259e-01]
 [-1.0246397e-01  2.0316161e-01  1.2053478e+00 ... -4.6703598e-01
  -1.1657699e-01 -5.8431464e-01]
 ...
 [-6.2457234e-01  5.4760098e-01  7.9778725e-01 ... -1.1421896e+00
  -5.5662388e-01  6.7024551e-02]
 [ 2.2457790e-01  4.6777037e-01  4.0461478e-01 ... -3.7099230e-01
   2.6795432e-01 -9.3467259e-01]
 [ 1.2760673e-01 -6.4966373e-02  9.0641159e-01 ... -2.3546387e-01
  -3.4886464e-01  1.4293572e-03]]
(88, 768)


In [ ]:
print(len(df))
print(type(len(df)))

89
<class 'int'>


In [ ]:
u = np.append(u, corpus_embeddings, axis = 0)

In [ ]:
print(u)
print(u.shape)

[[-5.4695594e-01  3.0213717e-01  1.4343424e+00 ... -1.8749633e-01
  -6.4143288e-01 -4.6065682e-01]
 [ 2.2457790e-01  4.6777037e-01  4.0461478e-01 ... -3.7099230e-01
   2.6795432e-01 -9.3467259e-01]
 [-1.0246397e-01  2.0316161e-01  1.2053478e+00 ... -4.6703598e-01
  -1.1657699e-01 -5.8431464e-01]
 ...
 [ 2.2457790e-01  4.6777037e-01  4.0461478e-01 ... -3.7099230e-01
   2.6795432e-01 -9.3467259e-01]
 [ 1.2760673e-01 -6.4966373e-02  9.0641159e-01 ... -2.3546387e-01
  -3.4886464e-01  1.4293572e-03]
 [ 3.5950491e-01  5.9129280e-01  5.0455487e-01 ... -5.3720915e-01
  -4.4278506e-02 -1.5489496e-01]]
(89, 768)


In [ ]:
cosine_sim = cosine_similarity(u)

In [ ]:
print(cosine_sim)
print(cosine_sim.shape)

[[0.99999964 0.91148424 0.90832657 ... 0.88893044 0.85914993 0.7882883 ]
 [0.91148424 0.99999994 0.9493128  ... 0.8642168  0.8537967  0.7486001 ]
 [0.90832657 0.9493128  1.0000001  ... 0.86457753 0.8685935  0.76341283]
 ...
 [0.88893044 0.8642168  0.86457753 ... 0.99999976 0.9216119  0.7414044 ]
 [0.85914993 0.8537967  0.8685935  ... 0.9216119  0.99999994 0.76168096]
 [0.7882883  0.7486001  0.76341283 ... 0.7414044  0.76168096 1.        ]]
(90, 90)


In [ ]:
question_index=len(df)
similar_questions = list(enumerate(cosine_sim[question_index]))

In [ ]:
print(similar_questions)

[(0, 0.7789553), (1, 0.77697), (2, 0.70415825), (3, 0.65682626), (4, 0.6665196), (5, 0.7024321), (6, 0.638595), (7, 0.7771417), (8, 0.7105598), (9, 0.7556522), (10, 0.84122723), (11, 0.738901), (12, 0.7080411), (13, 0.7876648), (14, 0.7511946), (15, 0.7581523), (16, 0.7199267), (17, 0.6805888), (18, 0.7338761), (19, 0.7040614), (20, 0.7141611), (21, 0.6179732), (22, 0.8086588), (23, 0.64564556), (24, 0.73328656), (25, 0.7345173), (26, 0.840538), (27, 0.80113524), (28, 0.6654245), (29, 0.7053929), (30, 0.8128777), (31, 0.7582425), (32, 0.7626153), (33, 0.7789681), (34, 0.80212426), (35, 0.78752434), (36, 0.7563779), (37, 0.7691169), (38, 0.7650509), (39, 0.65606827), (40, 0.694941), (41, 0.69817954), (42, 0.7410891), (43, 0.71316993), (44, 0.65872306), (45, 0.7016727), (46, 0.8137785), (47, 0.65575194), (48, 0.720351), (49, 0.8144953), (50, 0.7436936), (51, 0.64969426), (52, 0.7719743), (53, 0.6924851), (54, 0.71692383), (55, 0.6662154), (56, 0.77412903), (57, 0.6763332), (58, 0.6062593

In [ ]:
sorted_similar_questions = sorted(similar_questions, key=lambda x:x[1], reverse=True)

In [ ]:
print(sorted_similar_questions)

[(88, 1.0), (10, 0.84122723), (26, 0.840538), (71, 0.8182774), (49, 0.8144953), (46, 0.8137785), (80, 0.8137785), (87, 0.8137785), (30, 0.8128777), (22, 0.8086588), (34, 0.80212426), (27, 0.80113524), (59, 0.7924472), (13, 0.7876648), (35, 0.78752434), (60, 0.78403056), (75, 0.78403056), (84, 0.78403056), (33, 0.7789681), (0, 0.7789553), (7, 0.7771417), (76, 0.7771417), (78, 0.7771417), (1, 0.77697), (79, 0.77697), (86, 0.77697), (67, 0.7764347), (56, 0.77412903), (52, 0.7719743), (37, 0.7691169), (72, 0.7679452), (38, 0.7650509), (73, 0.7650509), (82, 0.7650509), (70, 0.76500475), (32, 0.7626153), (31, 0.7582425), (15, 0.7581523), (36, 0.7563779), (9, 0.7556522), (63, 0.75410944), (14, 0.7511946), (68, 0.7506178), (50, 0.7436936), (42, 0.7410891), (11, 0.738901), (64, 0.73575974), (25, 0.7345173), (18, 0.7338761), (24, 0.73328656), (48, 0.720351), (16, 0.7199267), (54, 0.71692383), (20, 0.7141611), (81, 0.7141611), (43, 0.71316993), (77, 0.71316993), (85, 0.71316993), (8, 0.7105598), 

In [ ]:
sorted_similar_questions.pop(0)

(88, 1.0)

In [ ]:
print(sorted_similar_questions)

[(10, 0.84122723), (26, 0.840538), (71, 0.8182774), (49, 0.8144953), (46, 0.8137785), (80, 0.8137785), (87, 0.8137785), (30, 0.8128777), (22, 0.8086588), (34, 0.80212426), (27, 0.80113524), (59, 0.7924472), (13, 0.7876648), (35, 0.78752434), (60, 0.78403056), (75, 0.78403056), (84, 0.78403056), (33, 0.7789681), (0, 0.7789553), (7, 0.7771417), (76, 0.7771417), (78, 0.7771417), (1, 0.77697), (79, 0.77697), (86, 0.77697), (67, 0.7764347), (56, 0.77412903), (52, 0.7719743), (37, 0.7691169), (72, 0.7679452), (38, 0.7650509), (73, 0.7650509), (82, 0.7650509), (70, 0.76500475), (32, 0.7626153), (31, 0.7582425), (15, 0.7581523), (36, 0.7563779), (9, 0.7556522), (63, 0.75410944), (14, 0.7511946), (68, 0.7506178), (50, 0.7436936), (42, 0.7410891), (11, 0.738901), (64, 0.73575974), (25, 0.7345173), (18, 0.7338761), (24, 0.73328656), (48, 0.720351), (16, 0.7199267), (54, 0.71692383), (20, 0.7141611), (81, 0.7141611), (43, 0.71316993), (77, 0.71316993), (85, 0.71316993), (8, 0.7105598), (12, 0.7080

In [ ]:
array = [46,80,87]
df.loc[df['row_num'].isin(array)]

,Unnamed: 0,original_index,q&a,q_fa_id,question,clusters,row_num
46,27911,27911,پدرم چند سال پیش فوت کرد و در وصیت نامه خود مق...,YNoiA38B7wsuAN6fbsJx,پدرم چند سال پیش فوت کرد و در وصیت نامه خود مق...,209,46
80,35949,35949,پدرم چند سال پیش فوت کرد و در وصیت نامه خود مق...,YduuBH8B7wsuAN6f-Hlu,پدرم چند سال پیش فوت کرد و در وصیت نامه خود مق...,209,80
87,38512,38512,پدرم چند سال پیش فوت کرد و در وصیت نامه خود مق...,odtkBX8B7wsuAN6fI6Qp,پدرم چند سال پیش فوت کرد و در وصیت نامه خود مق...,209,87


In [ ]:
def get_title_from_index(index):
    return df[df.index == index]["question"].values[0]
result=[]
i=0
for movie in sorted_similar_questions:
    q=get_title_from_index(movie[0])
    print(q)
    result.append(q)

    i=i+1
    if i>15:
        break


پدرم زمینی را به همسر اولش هبه کرد و پس از فوت او این زمین را به نام من سند زد که از همسر دوم ایشان می باشم. آیا این عمل مشکل شرعی دارد؟  
پدرم درباره مصرف یک سوم از اموالش وصیتی داشت و مادرم می‌گوید من وصی پدر هستم، آیا من می‌توانم سهم خود را از ثلث مطالبه کرده و با تشخیص خود صرف امور خیر کنم؟
پدر ما دارای یک قطعه زمین بود که در زمان حیاتش به سه قسمت تقسیم شده بود و با رضایت خودش دو قسمت از آن سه قسمت توسط دو نفر از پسران ایشان ساخته شد که در آن زندگی می کردند.می خواهم بدانم که آیا در تقسیم میراث متوفی ، هر سه قطعه زمین باید محاسبه شود یا فقط همان یک قسمت باقیمانده باید طبق شرع بین دختران و پسران تقسیم شود؟
در تصادفی که پدرم راننده و مقصر بود مادرم فوت کرد. دیه‌ای که بیمه خواهد داد به چه کسانی می‌رسد؟
پدرم چند سال پیش فوت کرد و در وصیت نامه خود مقدار دو میلیون تومان را جهت تهیه جهیزیه برای هر یک از دخترها در نظر گرفت. اکنون بعد از گذشت 13 سال اقدام به تقسیم اموال شده است؟ آیا  این پول به نرخ امروز در نظر گرفته می‌شود؟
پدرم چند سال پیش فوت کرد و در وصیت نامه خود مقدار دو میلیون تومان ر

In [ ]:
print(result)

['خمس سهام اهدایی به فرزند در زمان کودکی', 'سهم الارث فرزندی که از کودکی نزد پدر کار کرده', 'قرض ربوی دادن پدر به فرزندان از پول فرزند صغیر ', 'شخصی فوت کرده و سهمی از ارث او به پدرش می\u200cرسد. پدر با آنکه می\u200cداند به این اموال خمس تعلق گرفته، سهم خود را به عروس و نوه\u200cهایش می\u200cبخشد. آیا اکنون چیزی بر عهده پدر است؟', 'ارث بردن قاضی از پدرش که حکم اعدام او را صادر کرده', 'طلب حلالیت بابت خوردن بدون اجازه میوه دیگران در کودکی', 'دزدی از اموال پدر قبل از بلوغ', 'مسئول پرداخت اقساط وام تحصیلی دختر بعد از ازدواج ', 'حکم تهیّه ی جهیزیه دختر توسط پدر ', 'پس گرفتن باغی که جهت نصف کاری در اختیار فرزند قرار داده شده', 'وصیت به فرزند برای حج و انجام حج توسط فرزند به نیت خود', 'استفاده از منزل پدری با اظهار نارضایتی پدر', 'حق فسخ فرزند نسبت به مالی که پدرش به او هبه کرده و بعد فروخته', 'ارث زن از چاه و قنات', 'کسر مخارج بیماری متوفی از ماترکش ', 'تصرف پدر در مال پسری که به كما رفته براي مخارج بيماري فرزند']


In [ ]:
question_user_like = "خمس سهام اهدایی به فرزند در زمان کودکی"
def get_index_from_title(title):
    return df[df. == title]["index"].values[0]
movie_index = get_index_from_title(movie_user_likes)

In [ ]:
final = np.append(final, pooled, axis = 0)